In [3]:
import pulp

Delta = 1 #da capire se serve, per ora meglio lasciare a 1 converte MW in MW/h
zones = ["Z1", "Z2", "Z3"]
slices = list(range(8))           # 8 quarti d'ora (2 ore)
S_in_H = {0:[0,1,2,3], 1:[4,5,6,7]}

# Linee orientate (NTC)
lines = [("Z1","Z2"), ("Z2","Z3")]
Fmax = {(l,s): 100 for l in lines for s in slices}

def inc(z, l):
    a,b = l
    if z == a: return -1
    if z == b: return +1
    return 0

# -----------------------
# OFFERTE (QH, HR, BLOCCHI con MAR)
# -----------------------
# Offerte QH: (z, s, typ, id, qty, price)
offers_QH = [
    ("Z1",0,"sell","s1", 100, 0), #("Z1",0,"buy","d1", 15, 60),
    ("Z1",1,"sell","s2", 100, 0), #("Z1",1,"buy","d2", 15, 60),
    ("Z1",2,"sell","s3", 100, 0), #("Z1",2,"buy","d3", 15, 55),
    ("Z1",3,"sell","s4", 90, 0), #("Z1",3,"buy","d4", 15, 55),
    #("Z1",4,"sell","s5", 20, 40), ("Z1",4,"buy","d5", 10, 50),
    #("Z1",5,"sell","s6", 20, 40), ("Z1",5,"buy","d6", 10, 50),
    #("Z1",6,"sell","s7", 20, 45), ("Z1",6,"buy","d7", 10, 45),
    #("Z1",7,"sell","s8", 20, 45), ("Z1",7,"buy","d8", 10, 45),
#
    #("Z2",0,"sell","s9",  35, 22), ("Z2",0,"buy","d9",  10, 65),
    #("Z2",1,"sell","s10", 35, 22), ("Z2",1,"buy","d10", 10, 65),
    #("Z2",2,"sell","s11", 35, 26), ("Z2",2,"buy","d11", 10, 60),
    #("Z2",3,"sell","s12", 35, 26), ("Z2",3,"buy","d12", 10, 60),
    #("Z2",4,"sell","s13", 35, 30), ("Z2",4,"buy","d13", 10, 55),
    #("Z2",5,"sell","s14", 35, 30), ("Z2",5,"buy","d14", 10, 55),
    #("Z2",6,"sell","s15", 35, 34), ("Z2",6,"buy","d15", 10, 50),
    #("Z2",7,"sell","s16", 35, 34), ("Z2",7,"buy","d16", 10, 50),
#
    #("Z3",0,"sell","s17", 20, 28), ("Z3",0,"buy","d17", 10, 58),
    #("Z3",1,"sell","s18", 20, 28), ("Z3",1,"buy","d18", 10, 58),
    #("Z3",2,"sell","s19", 20, 30), ("Z3",2,"buy","d19", 10, 55),
    #("Z3",3,"sell","s20", 20, 30), ("Z3",3,"buy","d20", 10, 55),
    #("Z3",4,"sell","s21", 25, 33), ("Z3",4,"buy","d21", 10, 52),
    #("Z3",5,"sell","s22", 25, 33), ("Z3",5,"buy","d22", 10, 52),
    #("Z3",6,"sell","s23", 25, 36), ("Z3",6,"buy","d23", 10, 48),
    #("Z3",7,"sell","s24", 25, 36), ("Z3",7,"buy","d24", 10, 48),
]

# Offerte HR (una variabile per ora, profilo piatto)
offers_HR_sell = [
    #("Z1",0,"hr1", 40, 34),
    #("Z1",1,"hr2", 40, 38),
    #("Z2",0,"hr3", 50, 24),
    #("Z2",1,"hr4", 50, 30),
    #("Z3",0,"hr5", 30, 29),
    #("Z3",1,"hr6", 30, 31),
]
offers_HR_buy = [
    ("Z1",0,"hrb1", 100, 2000),
    #("Z1",1,"hrb2", 20, 49),
    #("Z2",0,"hrb3", 25, 61),
    #("Z2",1,"hrb4", 25, 53),
    #("Z3",0,"hrb5", 15, 56),
    #("Z3",1,"hrb6", 15, 51),
]

# Blocchi con MAR: vendite e acquisti
# vendita: (z, id, price, {slice: qty}, MAR)
blocks_sell = [
    #("Z1","B1", 33, {0:15,1:15,2:15,3:15}, 0.5),      # MAR 50%
    #("Z2","B2", 27, {4:20,5:20,6:20,7:20}, 0.6),
    #("Z3","B3", 32, {2:10,3:10,4:10}, 0.4),
]
# acquisto blocks
blocks_buy = [
    #("Z1","RB1", 55, {0:10,1:10,2:10,3:10}, 0.5),
    #("Z2","RB2", 52, {4:12,5:12,6:12,7:12}, 0.5),
]

# -----------------------
# BUILD MODEL (MILP o LP con blocchi fissati)
# -----------------------
def build_model(fix_blocks=None, lp_relax=False):
    """
    fix_blocks: dict {(z, oid): 0/1} per fissare u variabili dei blocchi (fase prezzi).
    lp_relax: se True, costruisce un LP (usa continuous per u), ma con MAR ancora applicati.
    """
    m = pulp.LpProblem("QH_HR_BLOCKS_MAR", pulp.LpMaximize)

    # QH vars
    x_QH = {}  # sells
    y_QH = {}  # buys
    for (z,s,typ,oid,qty,price) in offers_QH:
        if typ=="sell":
            x_QH[(z,s,oid)] = pulp.LpVariable(f"xQH_{oid}_{z}_{s}", lowBound=0, upBound=qty)
        else:
            y_QH[(z,s,oid)] = pulp.LpVariable(f"yQH_{oid}_{z}_{s}", lowBound=0, upBound=qty)

    # HR vars (one continuous var per hour offer)
    xH = {}
    yH = {}
    for (z,h,oid,qty,price) in offers_HR_sell:
        xH[(z,h,oid)] = pulp.LpVariable(f"xH_{oid}_{z}_{h}", lowBound=0, upBound=qty)
    for (z,h,oid,qty,price) in offers_HR_buy:
        yH[(z,h,oid)] = pulp.LpVariable(f"yH_{oid}_{z}_{h}", lowBound=0, upBound=qty)

    # Block vars:
    # uS / uB: binary (or fixed) => indicate acceptance
    # gammaS[(z,oid,s)] in [0,1]: fraction accepted of block's qty in slice s
    uS = {}
    gammaS = {}
    for (z, oid, price, prof, mar) in blocks_sell:
        if fix_blocks and (z, oid) in fix_blocks:
            val = 1 if fix_blocks[(z,oid)] >= 0.5 else 0
            uS[(z,oid)] = pulp.LpVariable(f"uS_{oid}_{z}", lowBound=val, upBound=val)
            # gamma bounds fixed accordingly below
        else:
            # binary in MILP, continuous in LP relaxation stage if lp_relax True
            if lp_relax:
                uS[(z,oid)] = pulp.LpVariable(f"uS_{oid}_{z}", lowBound=0, upBound=1)
            else:
                uS[(z,oid)] = pulp.LpVariable(f"uS_{oid}_{z}", cat="Binary")
        for s,q in prof.items():
            name = (z,oid,s)
            # gamma in [0,1], but bounds will be constrained by MAR and uS
            # If fix_blocks is provided and u fixed, we can tighten gamma bounds at creation
            if fix_blocks and (z,oid) in fix_blocks:
                val = 1 if fix_blocks[(z,oid)] >= 0.5 else 0
                low = mar * val
                high = val
                gammaS[name] = pulp.LpVariable(f"gammaS_{oid}_{z}_s{s}", lowBound=low, upBound=high)
            else:
                gammaS[name] = pulp.LpVariable(f"gammaS_{oid}_{z}_s{s}", lowBound=0, upBound=1)

    # buy blocks
    uB = {}
    gammaB = {}
    for (z, oid, valprice, prof, mar) in blocks_buy:
        if fix_blocks and (z, oid) in fix_blocks:
            val = 1 if fix_blocks[(z,oid)] >= 0.5 else 0
            uB[(z,oid)] = pulp.LpVariable(f"uB_{oid}_{z}", lowBound=val, upBound=val)
        else:
            if lp_relax:
                uB[(z,oid)] = pulp.LpVariable(f"uB_{oid}_{z}", lowBound=0, upBound=1)
            else:
                uB[(z,oid)] = pulp.LpVariable(f"uB_{oid}_{z}", cat="Binary")
        for s,q in prof.items():
            name = (z,oid,s)
            if fix_blocks and (z,oid) in fix_blocks:
                val = 1 if fix_blocks[(z,oid)] >= 0.5 else 0
                low = mar * val
                high = val
                gammaB[name] = pulp.LpVariable(f"gammaB_{oid}_{z}_s{s}", lowBound=low, upBound=high)
            else:
                gammaB[name] = pulp.LpVariable(f"gammaB_{oid}_{z}_s{s}", lowBound=0, upBound=1)

    # Flussi
    f = {(l,s): pulp.LpVariable(f"f_{l[0]}_{l[1]}_{s}", lowBound=-Fmax[(l,s)], upBound=Fmax[(l,s)])
         for l in lines for s in slices}

    # Obiettivo (welfare)
    obj = []
    # QH
    for (z,s,typ,oid,qty,price) in offers_QH:
        if typ=="sell":
            obj.append(- price * x_QH[(z,s,oid)] * Delta)
        else:
            obj.append(+ price * y_QH[(z,s,oid)] * Delta)
    # HR
    for (z,h,oid,qty,price) in offers_HR_sell:
        for s in S_in_H[h]:
            obj.append(- price * xH[(z,h,oid)] * Delta)
    for (z,h,oid,qty,price) in offers_HR_buy:
        for s in S_in_H[h]:
            obj.append(+ price * yH[(z,h,oid)] * Delta)
    # Blocks SELL: cost = -price * (qty * gamma)
    for (z, oid, price, prof, mar) in blocks_sell:
        for s,q in prof.items():
            obj.append(- price * q * gammaS[(z,oid,s)] * Delta)
    # Blocks BUY: value = +price * (qty * gamma)
    for (z, oid, price, prof, mar) in blocks_buy:
        for s,q in prof.items():
            obj.append(+ price * q * gammaB[(z,oid,s)] * Delta)

    m += pulp.lpSum(obj)

    # ------------------
    # Vincoli
    # ------------------
    # Relazioni MAR: gamma <= u and gamma >= MAR * u
    for (z, oid, price, prof, mar) in blocks_sell:
        for s,q in prof.items():
            m += gammaS[(z,oid,s)] <= uS[(z,oid)], f"gammaS_le_u_{oid}_{z}_s{s}"
            m += gammaS[(z,oid,s)] >= mar * uS[(z,oid)], f"gammaS_ge_mar_{oid}_{z}_s{s}"
    for (z, oid, price, prof, mar) in blocks_buy:
        for s,q in prof.items():
            m += gammaB[(z,oid,s)] <= uB[(z,oid)], f"gammaB_le_u_{oid}_{z}_s{s}"
            m += gammaB[(z,oid,s)] >= mar * uB[(z,oid)], f"gammaB_ge_mar_{oid}_{z}_s{s}"

    # Bilanci per zona e slice
    for z in zones:
        for s in slices:
            supply = (
                pulp.lpSum(x_QH[(zz,ss,oid)] for (zz,ss,oid) in x_QH if zz==z and ss==s)
                + pulp.lpSum(xH[(zz,h,oid)] for (zz,h,oid,qty,price) in offers_HR_sell if zz==z and s in S_in_H[h])
                + pulp.lpSum(q * gammaS[(zz,oid,ss)] for (zz,oid,price,prof,mar) in blocks_sell for (ss,q) in prof.items() if zz==z and ss==s)
            )
            demand = (
                pulp.lpSum(y_QH[(zz,ss,oid)] for (zz,ss,oid) in y_QH if zz==z and ss==s)
                + pulp.lpSum(yH[(zz,h,oid)] for (zz,h,oid,qty,price) in offers_HR_buy if zz==z and s in S_in_H[h])
                + pulp.lpSum(q * gammaB[(zz,oid,ss)] for (zz,oid,price,prof,mar) in blocks_buy for (ss,q) in prof.items() if zz==z and ss==s)
            )
            net_flow = pulp.lpSum(inc(z,l) * f[(l,s)] for l in lines)
            m += demand - supply - net_flow == 0, f"balance_{z}_s{s}"

    return m

# -----------------------
# FASE 1: MILP
# -----------------------
milp = build_model()
milp.solve(pulp.PULP_CBC_CMD(msg=False))
print("MILP status:", pulp.LpStatus[milp.status])
print("MILP objective (welfare):", round(pulp.value(milp.objective), 2))

# Recupero blocchi accettati (u valori) per fissarli nella fase LP
fixed_blocks = {}
# Cerca variabili uS_... e uB_...
for v in milp.variables():
    if v.name.startswith("uS_") or v.name.startswith("uB_"):
        parts = v.name.split("_")
        # uS_<oid>_<zone>  => parts[0]=uS, parts[1]=<oid>, parts[2]=<zone>
        oid = parts[1]
        z = parts[2]
        fixed_blocks[(z, oid)] = v.varValue
print("Blocchi (u) dagli interi MILP (per fase prezzi):", fixed_blocks)

# Mostra anche gamma effettive trovate nel MILP
print("\nGamma (frazione per blocchi) trovate dal MILP:")
for (z, oid, price, prof, mar) in blocks_sell:
    for s,q in prof.items():
        name = f"gammaS_{oid}_{z}_s{s}"
        try:
            val = [v.varValue for v in milp.variables() if v.name == name][0]
        except IndexError:
            val = None
        print(f"  Sell {oid} in {z} slice {s}: gamma={val}")
for (z, oid, price, prof, mar) in blocks_buy:
    for s,q in prof.items():
        name = f"gammaB_{oid}_{z}_s{s}"
        try:
            val = [v.varValue for v in milp.variables() if v.name == name][0]
        except IndexError:
            val = None
        print(f"  Buy  {oid} in {z} slice {s}: gamma={val}")

# -----------------------
# FASE 2: LP (blocchi fissati) per ottenere i prezzi (duali)
# -----------------------
lp = build_model(fix_blocks=fixed_blocks, lp_relax=True)
lp.solve(pulp.PULP_CBC_CMD(msg=False))
print("\nLP status:", pulp.LpStatus[lp.status])

# Estrazione prezzi (duali) dai vincoli balance_{z}_s{s}
print("\n=== Prezzi zonali (€/MWh) per ogni quart'ora ===")
for z in zones:
    for s in slices:
        cname = f"balance_{z}_s{s}"
        pi = lp.constraints[cname].pi
        print(f"  Zona {z}, slice {s}: price = {pi:.2f}")

# Stampa riepilogo quantità accettate e flussi (LP solution)
print("\n--- Riepilogo (LP solution) ---")
# raccolgo variabili di interesse
vars_lp = {v.name: v.varValue for v in lp.variables()}

# Flussi
print("\nFlussi (per linea e slice):")
for l in lines:
    for s in slices:
        vname = f"f_{l[0]}_{l[1]}_{s}"
        print(f"  {l} s{s}: {vars_lp.get(vname,0):8.2f}")

# Blocchi: gamma e u (fissati)
print("\nBlocchi sell (gamma) e u (fissati):")
for (z,oid,price,prof,mar) in blocks_sell:
    uval = fixed_blocks.get((z,oid), 0)
    print(f"  Block {oid} in {z}: u={uval}")
    for s,q in prof.items():
        gname = f"gammaS_{oid}_{z}_s{s}"
        print(f"    slice {s}: gamma={vars_lp.get(gname,0):6.3f} -> MW = {q * vars_lp.get(gname,0,):6.3f}")

print("\nBlocchi buy (gamma) and u (fissati):")
for (z,oid,price,prof,mar) in blocks_buy:
    uval = fixed_blocks.get((z,oid), 0)
    print(f"  Block {oid} in {z}: u={uval}")
    for s,q in prof.items():
        gname = f"gammaB_{oid}_{z}_s{s}"
        print(f"    slice {s}: gamma={vars_lp.get(gname,0):6.3f} -> MW = {q * vars_lp.get(gname,0,):6.3f}")

# Piccolo riepilogo offerte HR/QH accettate (LP)
print("\nOfferte HR (accepted MW per QH):")
for (z,h,oid,qty,price) in offers_HR_sell:
    v = vars_lp.get(f"xH_{oid}_{z}_{h}", 0)
    if v is not None:
        print(f"  Sell {oid} ({z},h{h}): {v:.2f} MW per QH")

print("\nOfferte QH (accepted MW):")
for (z,s,typ,oid,qty,price) in offers_QH:
    if typ=="sell":
        v = vars_lp.get(f"xQH_{oid}_{z}_{s}", 0)
        print(f"  Sell {oid} ({z},s{s}): {v:.2f} MW")
    else:
        v = vars_lp.get(f"yQH_{oid}_{z}_{s}", 0)
        print(f"  Buy  {oid} ({z},s{s}): {v:.2f} MW")


MILP status: Optimal
MILP objective (welfare): 720000.0
Blocchi (u) dagli interi MILP (per fase prezzi): {}

Gamma (frazione per blocchi) trovate dal MILP:

LP status: Optimal

=== Prezzi zonali (€/MWh) per ogni quart'ora ===
  Zona Z1, slice 0: price = -0.00
  Zona Z1, slice 1: price = -0.00
  Zona Z1, slice 2: price = -0.00
  Zona Z1, slice 3: price = 8000.00
  Zona Z1, slice 4: price = -0.00
  Zona Z1, slice 5: price = -0.00
  Zona Z1, slice 6: price = -0.00
  Zona Z1, slice 7: price = -0.00
  Zona Z2, slice 0: price = -0.00
  Zona Z2, slice 1: price = -0.00
  Zona Z2, slice 2: price = -0.00
  Zona Z2, slice 3: price = 8000.00
  Zona Z2, slice 4: price = -0.00
  Zona Z2, slice 5: price = -0.00
  Zona Z2, slice 6: price = -0.00
  Zona Z2, slice 7: price = -0.00
  Zona Z3, slice 0: price = -0.00
  Zona Z3, slice 1: price = -0.00
  Zona Z3, slice 2: price = -0.00
  Zona Z3, slice 3: price = 8000.00
  Zona Z3, slice 4: price = -0.00
  Zona Z3, slice 5: price = -0.00
  Zona Z3, slice 6: 